# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096979


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.41s/it]

Rendering models:  10%|█         | 3/30 [00:04<00:38,  1.43s/it]

Rendering models:  17%|█▋        | 5/30 [00:04<00:25,  1.02s/it]

Rendering models:  23%|██▎       | 7/30 [00:04<00:16,  1.37it/s]

...

Rendering models:  70%|███████   | 21/30 [00:07<00:02,  3.09it/s]

Rendering models:  77%|███████▋  | 23/30 [00:07<00:01,  4.08it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  5.01it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  6.66it/s]

not-logged-in-673e227f920b24558cc1    0.000313
Jnursssmith5263                       0.000471
DannyU                                0.000181
TateK                                 0.000400
jnarayanbvg                           0.000189
Jonas_Cross                           0.000375
not-logged-in-db23dc032fc427a4e4d9    0.000228
awright5                              0.001567
not-logged-in-74261f68b83c7ae82ce3    0.000181
lsautter                              0.000224
Zopin                                 0.016018
Synapsestar                           0.000303
dinosora                              0.000202
Lavadude                              0.002549
not-logged-in-4a99265f22da29adddc5    0.001072
not-logged-in-153517196e8c4276bfd0    0.003914
cooper.hull                           0.025659
not-logged-in-5992679fdf3537b5607f    0.025942
cheez__it                             0.000311
SatanSoo                              0.000542
Getrekt10127                          0.977019
h4ranek      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())